In [14]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, make_scorer
from fetch_hang_ADNI_data import load_for_perturbation, read_data
def prepare_data(all_data):
    """
    Prepare the data for SVM classification.
    
    Parameters:
        all_data (list): List of Data objects containing connectome information.
    
    Returns:
        X (np.ndarray): Array of node features.
        y (np.ndarray): Array of labels.
    """
    X = []
    y = []
    for data in all_data:
        node_features = data.x.numpy()
        label = data.y.item()
        upper_triangle_indices = np.triu_indices_from(node_features)
        upper_triangle_features = node_features[upper_triangle_indices]
        X.append(upper_triangle_features.flatten())  # Flatten the node features into a single array
        y.append(label)
    return np.array(X), np.array(y)

def sensitivity_specificity_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    cm = confusion_matrix(y, y_pred)
    TN, FP, FN, TP = cm.ravel()
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    return sensitivity, specificity

def evaluate_classification(all_data):
    X, y = prepare_data(all_data)
    
    # Standardize the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Initialize SVM classifier
    svm_classifier = SVC(kernel='linear', random_state=42)
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    sensitivity_scores = []
    specificity_scores = []
    accuracy_scores = []
    
    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        svm_classifier.fit(X_train, y_train)
        y_pred = svm_classifier.predict(X_test)
        
        cm = confusion_matrix(y_test, y_pred)
        TN, FP, FN, TP = cm.ravel()
        
        sensitivity = TP / (TP + FN)
        specificity = TN / (TN + FP)
        accuracy = accuracy_score(y_test, y_pred)
        
        sensitivity_scores.append(sensitivity)
        specificity_scores.append(specificity)
        accuracy_scores.append(accuracy)
    
    print(f"Cross-Validation Accuracy: {np.mean(accuracy_scores):.2f} ± {np.std(accuracy_scores):.2f}")
    print(f"Sensitivity: {np.mean(sensitivity_scores):.2f} ± {np.std(sensitivity_scores):.2f}")
    print(f"Specificity: {np.mean(specificity_scores):.2f} ± {np.std(specificity_scores):.2f}")
    
    return np.mean(accuracy_scores), np.mean(sensitivity_scores), np.mean(specificity_scores)

# Group pairs to evaluate
group_pairs = [
    ('CN', 'CN'),
    ('CN', 'Dementia'),
    ('CN', 'MCI'),
    ('MCI', 'Dementia')
]

# Main loop to process each group pair
#mat_files_dir = '/media/hang/EXTERNAL_US/Data/1_HANG_FDG_PET/ADNI_Second_organized/KDE_Results_corrected_by_age_sec_education'
mat_files_dir ="/media/hang/EXTERNAL_US/Data/1_HANG_FDG_PET/longitudinal_AD_MCI_CN/mixed_all_connectome"
baseline = True  # Set to True or False based on your requirement
percentile = 25
resample_data = True
connectomes = ['ScaledMahalanobisDistanceMatrix', 'Z_scoring', 'K_correlation', 'K_JS_Divergence']

for connectome in connectomes:
    for class_pair in group_pairs:
        print(f"Evaluating pair: {class_pair}")
        
        # all_data = load_for_perturbation(baseline=baseline, class_pair=class_pair, percentile=percentile, resample_data=resample_data)
        all_data = read_data(class_pair, mat_files_dir, connectome, 2, baseline=baseline, percentile=percentile, resample_data=resample_data)
        if not all_data:
            print(f"No valid data for class pair: {class_pair}")
            continue
        
        accuracy, sensitivity, specificity = evaluate_classification(all_data)
        print(f"Scores for {connectome} {class_pair}: Accuracy: {accuracy:.2f}, Sensitivity: {sensitivity:.2f}, Specificity: {specificity:.2f}")


# Note: Make sure you define `load_for_perturbation` and other necessary functions before running this script.


Evaluating pair: ('CN', 'CN')


/home/hang/GitHub/BrainGNN_Pytorch/fetch_hang_ADNI_data.py:91: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNI_merge = pd.read_csv('/media/hang/EXTERNAL_US/Data/1_HANG_FDG_PET/ADNIMERGE_30Jan2024.csv')


Baseline not identical to followup in subject 035_S_4256 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 072_S_4226 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 116_S_4898 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 128_S_4553 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 153_S_4838 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 006_S_4449 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 014_S_4093 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 020_S_5203 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 032_S_4429 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_4578 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_5287 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 127_S_4843 of ab long 0 and ab

/home/hang/GitHub/BrainGNN_Pytorch/fetch_hang_ADNI_data.py:91: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNI_merge = pd.read_csv('/media/hang/EXTERNAL_US/Data/1_HANG_FDG_PET/ADNIMERGE_30Jan2024.csv')


Baseline not identical to followup in subject 035_S_4256 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 072_S_4226 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 116_S_4898 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 128_S_4553 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 153_S_4838 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 006_S_4449 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 014_S_4093 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 020_S_5203 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 032_S_4429 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_4578 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_5287 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 127_S_4843 of ab long 0 and ab

...

Baseline not identical to followup in subject 035_S_4256 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 072_S_4226 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 116_S_4898 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 128_S_4553 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 153_S_4838 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 006_S_4449 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 014_S_4093 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 020_S_5203 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 032_S_4429 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_4578 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_5287 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 127_S_4843 of ab long 0 and ab

/home/hang/GitHub/BrainGNN_Pytorch/fetch_hang_ADNI_data.py:91: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNI_merge = pd.read_csv('/media/hang/EXTERNAL_US/Data/1_HANG_FDG_PET/ADNIMERGE_30Jan2024.csv')


Baseline not identical to followup in subject 035_S_4256 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 072_S_4226 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 116_S_4898 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 128_S_4553 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 153_S_4838 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 006_S_4449 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 014_S_4093 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 020_S_5203 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 032_S_4429 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_4578 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_5287 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 127_S_4843 of ab long 0 and ab

/home/hang/GitHub/BrainGNN_Pytorch/fetch_hang_ADNI_data.py:91: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  ADNI_merge = pd.read_csv('/media/hang/EXTERNAL_US/Data/1_HANG_FDG_PET/ADNIMERGE_30Jan2024.csv')


Baseline not identical to followup in subject 035_S_4256 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 072_S_4226 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 116_S_4898 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 128_S_4553 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 153_S_4838 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 006_S_4449 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 014_S_4093 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 020_S_5203 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 032_S_4429 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_4578 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 053_S_5287 of ab long 0 and ab bl [1]
Baseline not identical to followup in subject 127_S_4843 of ab long 0 and ab